In [ ]:
import cv2
import tensorflow as tf
import numpy as np
from pygame import mixer

mixer.init()
sound= mixer.Sound('alarm.WAV')

# loading the saved model    
new_model  = tf.keras.models.load_model('my_model.h5')

eyeCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot Open webcam")

counter = 0
    
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eyeCascade.detectMultiScale(gray,1.1,4)
    for x,y,w,h in eyes:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        eye = eyeCascade.detectMultiScale(roi_gray)
    if len(eye) == 0:
        print("Eyes are not detected")
    else:
        for (ex,ey,ew,eh) in eye:
            eye_roi = roi_color[ey:ey+eh, ex:ex+ew]
    
    final_image = cv2.resize(eye_roi,(224,224))
    final_image = np.expand_dims(final_image,axis=0)
    final_image = final_image/225.0
    
    prediction = new_model.predict(final_image)
    print(prediction)
    if(prediction > 0.015):
        status = "Open Eyes"
        cv2.putText(frame,status,(150,150),cv2.FONT_HERSHEY_SIMPLEX, 3,(0,255,0),2,cv2.LINE_4)
        x1,y1,w1,h1 = 0,0,175,75 # Draw a black background rectangle
        cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0,0,0),-1) # Add text
        cv2.putText(frame,'Active!!',(x1+int(w1/10),y1 + int(h1/2)),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),2)
    else:
        counter = counter + 1
        status = "Closed Eyes"
        cv2.putText(frame,status,(150,150),cv2.FONT_HERSHEY_SIMPLEX, 3,(0,0,255),2,cv2.LINE_4)
        cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0,0,0),2) # Add text
        if counter>50:
            x1,y1,w1,h1 = 0,0,175,75 # Draw a black background rectangle
            cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0,0,0),-1) # Add text
            cv2.putText(frame,'Sleep Alert !!',(x1+int(w1/10),y1 + int(h1/2)),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
            sound.play()
            counter = 0
           
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    print(faceCascade.empty())
    face = faceCascade.detectMultiScale(gray,1.1,4)
    
    
    # Draw a rectangle around the face
    
    for (x,y,w,h) in face:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    # use putText() method for inserting text on video
    
    #cv2.putText(frame,status,(50,50),font, 3,(0,0,255),2,cv2.LINE_4)
    cv2.imshow('Drowsiness Detection Tool', frame)
    
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
    